In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
churn = pd.read_csv('C:/Users/acerpc/Desktop/data_folder/telecom_churn.csv')
churn.head()

,subscriberID,churn,gender,AGE,edu_class,incomeCode,duration,feton,peakMinAv,peakMinDiff,posTrend,negTrend,nrProm,prom,curPlan,avgplan,planChange,posPlanChange,negPlanChange,call_10086
0,19164958.0,1.0,0.0,20.0,2.0,12.0,16.0,0.0,113.666667,-8.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,39244924.0,1.0,1.0,20.0,0.0,21.0,5.0,0.0,274.000000,-371.0,0.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,0.0,1.0
2,39578413.0,1.0,0.0,11.0,1.0,47.0,3.0,0.0,392.000000,-784.0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,1.0
3,40992265.0,1.0,0.0,43.0,0.0,4.0,12.0,0.0,31.000000,-76.0,0.0,1.0,2.0,1.0,3.0,3.0,0.0,0.0,0.0,1.0
4,43061957.0,1.0,1.0,60.0,0.0,9.0,14.0,0.0,129.333333,-334.0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.model_selection import train_test_split
x = churn.loc[:,'gender':]
y = churn['churn']
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.3,random_state=42)

In [10]:
from sklearn.ensemble import BaggingClassifier

bgc = BaggingClassifier()
bgc.fit(train_x,train_y)
bgc.predict_proba(test_x)[:,1]

array([0.3, 0. , 0. , ..., 0.6, 0.9, 0.6])

In [11]:
from sklearn.metrics import classification_report
print(classification_report(test_y,bgc.predict(test_x)))

              precision    recall  f1-score   support

         0.0       0.80      0.89      0.85       557
         1.0       0.86      0.75      0.80       482

   micro avg       0.83      0.83      0.83      1039
   macro avg       0.83      0.82      0.82      1039
weighted avg       0.83      0.83      0.82      1039



In [13]:
from sklearn.model_selection import cross_val_score
cross_val_score(bgc,train_x,train_y,cv=5,scoring='roc_auc')

array([0.92240414, 0.907109  , 0.92453076, 0.94246611, 0.9066041 ])

In [15]:
churn['churn'].value_counts()

0.0    1929
1.0    1534
Name: churn, dtype: int64

非均衡样本

In [16]:
data = pd.read_csv('C:/Users/acerpc/Desktop/data_folder/train_modified.csv')
data.head()

,Disbursed,Existing_EMI,ID,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0,0.0,ID000002C20,300000,5,20000,1,0,37,1,...,0,0,0,0,1,1,0,1,0,0
1,0,0.0,ID000004E40,200000,2,35000,3,13,30,0,...,0,0,0,0,1,0,1,1,0,0
2,0,0.0,ID000007H20,600000,4,22500,1,0,34,1,...,0,0,0,0,0,0,1,0,0,1
3,0,0.0,ID000008I30,1000000,5,35000,3,10,28,1,...,0,0,0,0,0,0,1,0,0,1
4,0,25000.0,ID000009J40,500000,2,100000,3,17,31,1,...,0,0,0,0,0,0,1,0,0,1


In [17]:
data.Disbursed.value_counts()

0    19680
1      320
Name: Disbursed, dtype: int64

In [20]:
x_train,x_test,y_train,y_test = train_test_split(data.drop(['Disbursed','ID'],axis=1),data.Disbursed,test_size=0.3,random_state=6)

In [22]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100,max_depth=10,min_samples_leaf=30,min_samples_split=130,oob_score=True,class_weight={0:1,1:50})
rfc.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 50},
            criterion='gini', max_depth=10, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=30,
            min_samples_split=130, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=None, oob_score=True,
            random_state=None, verbose=0, warm_start=False)

In [24]:
from sklearn.metrics import accuracy_score,roc_auc_score
accuracy_score(y_test,rfc.predict(x_test))

0.8498333333333333

In [28]:
from sklearn import metrics
roc_auc_score(y_test,rfc.predict_proba(x_test))
metrics.recall_score(y_test,rfc.predict(x_test))

0.5625

In [30]:
roc_auc_score(y_test,rfc.predict_proba(x_test)[:,1])

0.8179147611788617

In [34]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': list(range(50,150)),'max_depth':list(range(10,21)),'min_samples_leaf':list(range(20,50)),'min_samples_split':list(range(80,151)),'class_weight':[{0:1,1:x} for x in range(101)]}
rf = GridSearchCV(RandomForestClassifier(oob_score=True),param_grid,scoring='roc_auc',cv=5)

In [ ]:
rf.fit(x_train,y_train)